In [1]:
import numpy as np
from pymatgen.core import Structure, Lattice, PeriodicSite, DummySpecie
from pymatgen.analysis.graphs import StructureGraph
from pymatgen.analysis.local_env import MinimumDistanceNN, CrystalNN 

In [2]:
def main():    
    # Load defective structure
    defective_struct = Structure.from_file("high_density_defects/MoS2_500/highMoS2cifs/MoS2_Mo56W4Se4S117_9df8176f-1a41-4d9d-a2e3-2d4b0aac5937.cif")
    struct_lattice = defective_struct.lattice

    # Get reference structure
    ref_unit_cell = Structure.from_file("high_density_defects/MoS2_500/MoS2.cif")
    reference_struct = ref_unit_cell.make_supercell([8,8,1])

    # Convert structures to dictionaries
    defective_dict = struct_to_dict(defective_struct)
    reference_dict = struct_to_dict(reference_struct)

    # Get defect structure
    defect_struct = get_defect_structure(defective_dict, reference_dict, struct_lattice)
    print(defect_struct)
    

def struct_to_dict(structure):
    list_of_sites = structure.sites
    list_of_frac_coords = np.round(structure.frac_coords,3)
    structure_dict = {i: j for i, j in zip(list_of_sites, list_of_frac_coords)}
    return structure_dict


def get_defect_structure(defective_dict, reference_dict, struct_lattice): 
    defect_site = []

    for ref_site, ref_coords in reference_dict.items():
        matching = False
        for def_site, def_coords in defective_dict.items():
            if np.array_equal(ref_coords, def_coords):
                matching = True
                if ref_site.specie != def_site.specie:  # Substitution case
                    defect_site.append(PeriodicSite(
                        species= def_site.species,
                        coords= def_site.frac_coords,
                        coords_are_cartesian=False,
                        lattice= struct_lattice,
                        properties= {"original_new_am": (ref_site.specie.Z, def_site.specie.Z)}
                    ))

        if not matching:           # Vacancy case
            defect_site.append(PeriodicSite(
                species=DummySpecie(),
                coords = ref_coords,
                coords_are_cartesian=False,
                lattice = struct_lattice,
                properties = {"original_new_am": (ref_site.specie.Z, 0)}
                ))

    # Create a structure with the defect sites
    defect_struct = Structure.from_sites(defect_site)
    return defect_struct


if __name__ == "__main__":
    main()

/home/adamkim/anaconda3/envs/sample001/lib/python3.13/site-packages/pymatgen/core/structure.py:3087: EncodingWarning: We strongly encourage explicit `encoding`, and we would use UTF-8 by default as per PEP 686
  with zopen(filename, mode="rt", errors="replace") as file:
/home/adamkim/anaconda3/envs/sample001/lib/python3.13/site-packages/pymatgen/io/cif.py:1290: UserWarning: Issues encountered while parsing CIF: 47 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/home/adamkim/anaconda3/envs/sample001/lib/python3.13/site-packages/pymatgen/io/cif.py:1290: UserWarning: Issues encountered while parsing CIF: 6 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


Full Formula (X11 W4 Se4)
Reduced Formula: X11(WSe)4
abc   :  25.522526  25.522526  20.000000
angles:  90.000000  90.000000 120.000000
pbc   :       True       True       True
Sites (19)
  #  SP           a         b         c  original_new_am
---  ----  --------  --------  --------  -----------------
  0  W     0.041667  0.958333  0.185988  (42, 74)
  1  X0+   0.167     0.458     0.186     (42, 0)
  2  X0+   0.417     0.083     0.186     (42, 0)
  3  W     0.416667  0.708333  0.185988  (42, 74)
  4  X0+   0.417     0.833     0.186     (42, 0)
  5  W     0.541667  0.333333  0.185988  (42, 74)
  6  W     0.666667  0.458333  0.185988  (42, 74)
  7  X0+   0.917     0.958     0.186     (42, 0)
  8  X0+   0.208     0.167     0.108     (16, 0)
  9  Se    0.208333  0.416667  0.107743  (16, 34)
 10  X0+   0.208     0.667     0.108     (16, 0)
 11  Se    0.333333  0.541667  0.107743  (16, 34)
 12  X0+   0.833     0.417     0.108     (16, 0)
 13  X0+   0.208     0.042     0.264     (16, 0)
 14  

In [ ]:
# What can you do with the defect structure
print(defect_struct)